In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
import logging
import inspect
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python3


### Get recording info (google sheets)

In [4]:
import requests
import pandas as pd
from io import BytesIO

In [5]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [6]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate,username,n_ephys_streams,max_video_duration_m
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10


### Submit job

In [7]:
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240808-3d-pipeline"

In [8]:
from multicamera_airflow_pipeline.tim_240731.interface.o2 import O2Runner
from pathlib import Path
from datetime import datetime

INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Python interpreter binary location: /n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python3
/n/groups/datta/tim_sainburg/conda_envs/peromoseq/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/n/groups/datta/tim_sainburg/conda_envs/peromoseq/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [16]:
import logging
import yaml
logger = logging.getLogger(__name__)

In [17]:
for idx, recording_row in recording_df.iterrows():
    break

In [27]:
# these are arguments
#output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240806-3d-pipeline"
output_directory = "/n/groups/datta/kpts_pipeline/tim_240731/results"
job_directory = Path('/n/groups/datta/tim_sainburg/datasets/scratch/jobs')
job_directory.mkdir(exist_ok=True, parents=True)
config_file = Path("/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/default_config.yaml")

In [28]:
output_directory = Path(output_directory)

In [29]:
def convert_minutes_to_hms(minutes_float):
    # Convert minutes to total seconds
    total_seconds = int(minutes_float * 60)

    # Extract hours, minutes, and seconds using divmod
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)

    # Format as HH:MM:SS
    return f"{hours:02}:{minutes:02}:{seconds:02}"

def check_camera_sync_completion(output_directory):
    return (output_directory / "camera_sync.csv").exists()


In [30]:
# load config
config_file = Path(config_file)
config = yaml.safe_load(open(config_file, "r"))

# where to save output
output_directory_camera_sync = (
    output_directory / "camera_sync" / recording_row.video_recording_id
)
logger.info("Starting sync cameras")

# check if sync is already completed
if config["sync_cameras"]["recompute_completed"]:
    if check_camera_sync_completion(output_directory_camera_sync):
        logger.info("Camera sync already completed, quitting")
        pass
    else:
        logger.info("Camera sync not completed, running")

# where the video data is located
recording_directory = (
    Path(recording_row.video_location_on_o2) / recording_row.video_recording_id
)

output_directory_camera_sync.mkdir(parents=True, exist_ok=True)
current_datetime_str = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
remote_job_directory = job_directory / current_datetime_str

# get camera info
samplerate = recording_row.samplerate
# trigger_pin = recording_row.trigzger_pin
params = {
    "recording_directory": recording_directory.as_posix(),
    "output_directory_camera_sync": output_directory_camera_sync.as_posix(),
    "samplerate": samplerate,
    # "trigger_pin": trigger_pin,
}

# specify the duration of the job (here, it should be short)
#  in config, set o2_runtime_multiplier to choose how long the job should
#  run relative to recording duration
duration_requested = convert_minutes_to_hms(
    recording_row.duration_m * config["o2"]["sync_cameras"]["o2_runtime_multiplier"]
)


INFO:__main__:Starting sync cameras


In [31]:
check_camera_sync_completion(output_directory_camera_sync)

True

In [21]:
check_camera_sync_completion(output_directory_camera_sync)

True